In [51]:
import sys
!{sys.executable} -m pip install matplotlib seaborn scikit-learn joblib




In [52]:
import sys
!{sys.executable} -m pip install pandas


In [53]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report



In [54]:
# Step 1: Load dataset
df = pd.read_csv("first_aid.csv")

# Show basic info
df.head()





,Description,Severity,First Aid Steps,Injury Type,Pill Needed
0,i have stomach pain after eating street food,MILD,"Drink clean water, rest, eat light food, if pe...",Abdominal Pain,antacid
1,severe stomach pain with vomiting and dizziness,SEVERE,"Lie down, hydrate if possible, seek immediate ...",Abdominal Pain,antacid
2,i have stomach cramps after eating too fast,MILD,"Sit upright, sip warm water, eat slowly next t...",Abdominal Pain,antacid
3,severe abdominal pain after eating old food,SEVERE,"Lie down, do not eat or drink until doctor con...",Abdominal Pain,antacid
4,my stomach hurts badly after roadside food,MODERATE,"Sip ORS slowly, rest, seek doctor if pain incr...",Abdominal Pain,antacid


In [55]:
df.info

<bound method DataFrame.info of                                          Description  Severity  \
0       i have stomach pain after eating street food      MILD   
1    severe stomach pain with vomiting and dizziness    SEVERE   
2        i have stomach cramps after eating too fast      MILD   
3        severe abdominal pain after eating old food    SEVERE   
4         my stomach hurts badly after roadside food  MODERATE   
..                                               ...       ...   
491                           how to cure a vertigo?      MILD   
492                        how do you treat a wound?      MILD   
493                     what to do if i get a wound?      MILD   
494         which medicine to take if i get wounded?      MILD   
495                             how to cure a wound?      MILD   

                                       First Aid Steps     Injury Type  \
0    Drink clean water, rest, eat light food, if pe...  Abdominal Pain   
1    Lie down, hydrate if p

In [56]:
df.describe()

,Description,Severity,First Aid Steps,Injury Type,Pill Needed
count,496,496,496,496,496
unique,492,3,330,52,13
top,severe nosebleed not stopping after 20 mins,MILD,Seek emergency medical care immediately.,miscellaneus case,NO
freq,2,227,14,135,387


In [57]:
# Step 2: Define input and output
X_raw = df.iloc[:, 0]          # First column (input text)
y = df.iloc[:, 1:]             # All other columns (outputs)


In [58]:
#Step 3: Clean text
def clean_text(text):
    text = str(text).lower()                          # lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text)           # remove special chars
    text = re.sub(r'\s+', ' ', text).strip()          # remove extra spaces
    return text
 
X_cleaned = X_raw.apply(clean_text)

# Step 4: Vectorize
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_cleaned)




In [59]:
X_cleaned.sample()

X_raw.sample()


307    mild feeling of tiredness after exam
Name: Description, dtype: object

In [60]:
X_raw.info()
X_cleaned.head()

<class 'pandas.core.series.Series'>
RangeIndex: 496 entries, 0 to 495
Series name: Description
Non-Null Count  Dtype 
--------------  ----- 
496 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


0       i have stomach pain after eating street food
1    severe stomach pain with vomiting and dizziness
2        i have stomach cramps after eating too fast
3        severe abdominal pain after eating old food
4         my stomach hurts badly after roadside food
Name: Description, dtype: object

In [61]:
y.info()
y.sample()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Severity         496 non-null    object
 1   First Aid Steps  496 non-null    object
 2   Injury Type      496 non-null    object
 3   Pill Needed      496 non-null    object
dtypes: object(4)
memory usage: 15.6+ KB


,Severity,First Aid Steps,Injury Type,Pill Needed
111,MILD,Check responsiveness and breathing. Call emerg...,CPR,NO


In [62]:
X

X_cleaned

0         i have stomach pain after eating street food
1      severe stomach pain with vomiting and dizziness
2          i have stomach cramps after eating too fast
3          severe abdominal pain after eating old food
4           my stomach hurts badly after roadside food
                            ...                       
491                              how to cure a vertigo
492                           how do you treat a wound
493                        what to do if i get a wound
494            which medicine to take if i get wounded
495                                how to cure a wound
Name: Description, Length: 496, dtype: object

In [63]:
y

,Severity,First Aid Steps,Injury Type,Pill Needed
0,MILD,"Drink clean water, rest, eat light food, if pe...",Abdominal Pain,antacid
1,SEVERE,"Lie down, hydrate if possible, seek immediate ...",Abdominal Pain,antacid
2,MILD,"Sit upright, sip warm water, eat slowly next t...",Abdominal Pain,antacid
3,SEVERE,"Lie down, do not eat or drink until doctor con...",Abdominal Pain,antacid
4,MODERATE,"Sip ORS slowly, rest, seek doctor if pain incr...",Abdominal Pain,antacid
...,...,...,...,...
491,MILD,Sit or lie down immediately to prevent falls. ...,Vertigo,Betahistine
492,MILD,Rinse the cut or wound with water and apply pr...,Wound,NO
493,MILD,Rinse the cut or wound with water and apply pr...,Wound,NO
494,MILD,Rinse the cut or wound with water and apply pr...,Wound,NO


In [64]:
X_cleaned.shape , y.shape

((496,), (496, 4))

In [77]:
# Step 5: Split train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42)




In [78]:
# Step 6: Create and train model
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42
)



In [3]:
import joblib
model = joblib.load("firstAIDmodel.pkl")
print(type(model))
print(model)


<class 'sklearn.pipeline.Pipeline'>
Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


In [5]:
import joblib

# Load the saved model
model = joblib.load("firstAIDmodel.pkl")

# Sample test input (must be a list of strings)
test_input = [
    "iam feeling dizzy"
]

# Make predictions
predictions = model.predict(test_input)

# If your labels are strings and you have a label binarizer, convert back to text
try:
    label_binarizer = joblib.load("label_binarizer.pkl")  # if you saved it separately
    decoded_predictions = label_binarizer.inverse_transform(predictions)
    print("Predictions:", decoded_predictions)
except FileNotFoundError:
    print("Predictions (raw):", predictions)



Predictions (raw): [['MILD'
  'Move to fresh air immediately, loosen clothing, seek medical help.'
  'miscellaneus case' 'NO']]
